## EDA:

Today we're looking at tweet data to find sentiment towards a product or brand.

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
from sklearn.manifold import TSNE
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from  nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Import Twitter Data
data = pd.read_csv('Data/Twitter_sentiment.csv', encoding='latin1')
data

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [3]:
data = data.rename(columns={"tweet_text": "Tweet", "emotion_in_tweet_is_directed_at": "Subject_of_tweet",
                     "is_there_an_emotion_directed_at_a_brand_or_product": "Emotion"})
data

,Tweet,Subject_of_tweet,Emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [4]:
#Looking for null values
data.isna().sum()

Tweet                  1
Subject_of_tweet    5802
Emotion                0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
Tweet               9092 non-null object
Subject_of_tweet    3291 non-null object
Emotion             9093 non-null object
dtypes: object(3)
memory usage: 213.2+ KB


In [6]:
data.describe()

,Tweet,Subject_of_tweet,Emotion
count,9092,3291,9093
unique,9065,9,4
top,RT @mention Marissa Mayer: Google Will Connect...,iPad,No emotion toward brand or product
freq,5,946,5389


In [7]:
#Taking a look at the null values to see the content
for i in range(9089, 9092):
    print('.............')
    print(data['Tweet'].iloc[i])

.............
Wave, buzz... RT @mention We interrupt your regularly scheduled #sxsw geek programming with big news {link}  #google #circles
.............
Google's Zeiger, a physician never reported potential AE. Yet FDA relies on physicians. &quot;We're operating w/out data.&quot; #sxsw #health2dev
.............
Some Verizon iPhone customers complained their time fell back an hour this weekend.  Of course they were the New Yorkers who attended #SXSW.


In [8]:
#Changing the data type to String so it can be passed through the cleaning function
data['Tweet'] = data['Tweet'].astype('str')

In [9]:
data['Tweet']

0       .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1       @jessedee Know about @fludapp ? Awesome iPad/i...
2       @swonderlin Can not wait for #iPad 2 also. The...
3       @sxsw I hope this year's festival isn't as cra...
4       @sxtxstate great stuff on Fri #SXSW: Marissa M...
                              ...                        
9088                        Ipad everywhere. #SXSW {link}
9089    Wave, buzz... RT @mention We interrupt your re...
9090    Google's Zeiger, a physician never reported po...
9091    Some Verizon iPhone customers complained their...
9092    Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...
Name: Tweet, Length: 9093, dtype: object

In [10]:
#Tokenizing the tweets and removing stopwords
# tokens = word_tokenize(data['Tweet'])

def tokenize(tweet):
    joined_tweet = ' '.join(tweet)
    tokens = joined_tweet.split(" ")
    
    return tokens
tokens = tokenize(data['Tweet'])
tokens

['.@wesley83',
 'I',
 'have',
 'a',
 '3G',
 'iPhone.',
 'After',
 '3',
 'hrs',
 'tweeting',
 'at',
 '#RISE_Austin,',
 'it',
 'was',
 'dead!',
 '',
 'I',
 'need',
 'to',
 'upgrade.',
 'Plugin',
 'stations',
 'at',
 '#SXSW.',
 '@jessedee',
 'Know',
 'about',
 '@fludapp',
 '?',
 'Awesome',
 'iPad/iPhone',
 'app',
 'that',
 "you'll",
 'likely',
 'appreciate',
 'for',
 'its',
 'design.',
 'Also,',
 "they're",
 'giving',
 'free',
 'Ts',
 'at',
 '#SXSW',
 '@swonderlin',
 'Can',
 'not',
 'wait',
 'for',
 '#iPad',
 '2',
 'also.',
 'They',
 'should',
 'sale',
 'them',
 'down',
 'at',
 '#SXSW.',
 '@sxsw',
 'I',
 'hope',
 'this',
 "year's",
 'festival',
 "isn't",
 'as',
 'crashy',
 'as',
 'this',
 "year's",
 'iPhone',
 'app.',
 '#sxsw',
 '@sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 '#SXSW:',
 'Marissa',
 'Mayer',
 '(Google),',
 'Tim',
 "O'Reilly",
 '(tech',
 'books/conferences)',
 '&amp;',
 'Matt',
 'Mullenweg',
 '(Wordpress)',
 '@teachntech00',
 'New',
 'iPad',
 'Apps',
 'For',
 '#SpeechTher

In [11]:
lemmatizer = WordNetLemmatizer()

In [12]:
tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens if w not in stopwords_list]

NameError: name 'stopwords_list' is not defined

In [ ]:
freqdist = FreqDist(tokens)

# get the 200 most common words 
most_common = freqdist.most_common(200)
most_common_10 = freqdist.most_common(10)

In [ ]:
most_common

In [ ]:
#Creating Stop words and removing puncuation
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['{link}', '@mention', '', 'i']

In [ ]:
stopwords_list

In [ ]:
len(tokens)

In [ ]:
len(stopped_tokens)

In [ ]:
df = pd.DataFrame(most_common_10, columns=["x", "y"])

In [ ]:
df

In [ ]:
g = sns.barplot(data=df, x="x", y="y")
plt.xticks(rotation=70)
plt.tight_layout()

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [ ]:
tweet_finder= BigramCollocationFinder.from_words(stopped_tokens)

tweet_score = tweet_finder.score_ngrams(bigram_measures.raw_freq)